In [1]:
#re-imports modules that have changed
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [73]:
import os
import logging
import json
import datetime as dt
import time
import pandas as pd
from src.candidate_fetcher import VoterInfo
from src.utils import CloudStorageClient

In [4]:
logging.basicConfig(level="INFO")

In [6]:
# Constants

In [30]:
# retrieve county level address data and set fips code as the identifer 

In [71]:
def run_voter_info(): 
    
    """
    Cloud function to run job to fetch voter information data for current elections
    from Google Civic Information API.
    Retrieves county level data and sets the fips code as the identifier (`geo_id`)
    Stores data in a cloud storage bucket as a json file.
    """
    
    # Job status
    logging.info("Starting job fetch voter information")
    #logging.info("""Trigger: messageId {} published at {}""".format(context.event_id, context.timestamp))
    
    # Initiate job
    civic = VoterInfo() 
    
    # Load elections, address data
    logging.info("Load list of current elections.")
    elections = civic.load_current_elections("current_elections",  "current_elections.json")
    
    logging.info("Load addreses by locale")
    locales = civic.load_address_locales("address_locales",  "addresses_county.csv")
    
    # Retrieve Voter information data
    for election in elections[0:2]:
        election_id = election['id']
        election_name = election['name']
        election_ocdid = election['ocdDivisionId']
        
        # Get state abbr from OCDid
        election_ocdid = election_ocdid.split("/")[-1].split(":")[-1].upper()

        logging.debug(f"election_id: {election_id}")
        logging.debug(f"election_ocdid: {election_ocdid}")
        logging.debug(f"election_name: {election_name}")
        
        # Subset data by OCDid
        # Except test election 
        if election_name == 'VIP Test Election': 
            continue
        # If election is national, return data for all records
        elif election_ocdid == 'US':
            active = locales.copy()
        # If election is statewide, return data for all records in state. 
        else: 
            active = locales.loc[locales['state_abbr'] == election_ocdid, :].copy()
        # Ensure active elections not null
        try: 
            assert(active.empty == False)
        except Exception as e: 
            logging.error("Unable to subset data by OCDid.")
    
        # Get voter information for election
        try: 
            logging.info(f"Start election: {election_id}:{election_ocdid}") 
            for index, row in active.iterrows():
                address = row['address']
                geo_id = row["fips"]
                response = civic.fetch_voter_info(address, election_id)
                response['geoid'] = {"fips":geo_id}
                civic.save_voter_info(geo_id, response, bucket_name="current_contests")
                time.sleep(1)
            logging.debug(f"Completed election: {election_id}:{election_ocdid}")
        except Exception as error: 
            logging.error(f"Failed to retrieve data for {election_id}:{election_ocdid}")
            logging.error(error)
    
    # Job status
    logging.info("Completed job fetch voter info.")

In [74]:
run_voter_info()

INFO:root:Starting job fetch voter information
INFO:root:Load list of current elections.
INFO:root:Successfully loaded current elections data.
INFO:root:Load addreses by locale
INFO:root:Successfully loaded address lookup data.
INFO:root:Start election: 4897:SC
ERROR:root:Successfully saved data for 45001 to: gs://current_contests/45001_2020-02-20.json
ERROR:root:Successfully saved data for 45003 to: gs://current_contests/45003_2020-02-20.json
ERROR:root:Successfully saved data for 45005 to: gs://current_contests/45005_2020-02-20.json
ERROR:root:Successfully saved data for 45007 to: gs://current_contests/45007_2020-02-20.json
ERROR:root:Successfully saved data for 45009 to: gs://current_contests/45009_2020-02-20.json
ERROR:root:Successfully saved data for 45011 to: gs://current_contests/45011_2020-02-20.json
ERROR:root:Successfully saved data for 45013 to: gs://current_contests/45013_2020-02-20.json
ERROR:root:Successfully saved data for 45015 to: gs://current_contests/45015_2020-02-20